In [ ]:
import os
import pandas as pd
import numpy as np
# Tensorflow library will requiere docker
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
# Tqdm library to show progress bar
from tqdm import tqdm

In [ ]:
def mobilenet_detector(path):
  """
  Builds a dataframe from mobilenet results from a folder with images
  Params: a string
  Returns: a dataframe
  """
  #initializethe model to predict the image details using predefined models.
  model = tf.keras.applications.MobileNetV3Small()

  folder_name = path

  # Creates a list of directories for each image
  directories = []
  for path, subdirs, files in os.walk(folder_name):
    for name in files:
      directories.append(os.path.join(path, name))

  # Creates an empty dataframe
  df = pd.DataFrame()

  # Creates columns
  img_name = []
  img_path = []
  img_folder_path = []
  target = []
  first_top_guess = []
  first_score = []
  second_top_guess = []
  second_score = []
  third_top_guess = []
  third_score = []

  # List of ImageNet classes that are considered cars
  car = ['ambulance', 'beach_wagon', 'cab', 'convertible', 'jeep', 'limousine', 'minivan', 'Model_T', 'racer', 'sports_car']

  # Applys the model to each image path
  for file_name in tqdm(directories):
    # Resize the image
    img = image.load_img(file_name,target_size=(224,224))
    resizedimg = image.img_to_array(img)
    finalimg = np.expand_dims(resizedimg,axis=0)
    finalimg.shape
    # Predict images
    predictions = model.predict(finalimg)
    results = imagenet_utils.decode_predictions(predictions, top=3)
    # Appends each result in the dataframe
    img_name.append(file_name.split('/')[4])
    img_folder_path.append(file_name.split('/')[3])
    img_path.append(file_name)
    first_top_guess.append(results[0][0][1:2][0])
    first_score.append(results[0][0][2:3][0])
    # Condition to create the target column.
    # If the top_guess is in the list and if the first_score is greater than 0.5,
    # then the target column will be appended with a 1
    if ((results[0][0][1:2][0] in car) and (results[0][0][2:3][0] > 0.5)):
      target.append(1)
    else:
      target.append(0)
    second_top_guess.append(results[0][1][1:2][0])
    second_score.append(results[0][1][2:3][0])
    third_top_guess.append(results[0][2][1:2][0])
    third_score.append(results[0][2][2:3][0])

  df['img_name'] = img_name
  df['img_path'] = img_path
  df['img_folder_path'] = img_folder_path
  df['target'] = target
  df['first_top_guess'] = first_top_guess
  df['first_score'] = first_score
  df['second_top_guess'] = second_top_guess
  df['second_score'] = second_score
  df['third_top_guess'] = third_top_guess
  df['third_score'] = third_score

  return df

In [ ]:
# Calls the function input must be a folder path
df = mobilenet_detector('/content/data')
df

  0%|          | 0/6 [00:00<?, ?it/s]

1/1 [==============================] - 6s 6s/step


 17%|█▋        | 1/6 [00:06<00:31,  6.36s/it]

1/1 [==============================] - 0s 20ms/step


 50%|█████     | 3/6 [00:06<00:05,  1.70s/it]

1/1 [==============================] - 0s 21ms/step


 83%|████████▎ | 5/6 [00:06<00:00,  1.16it/s]

1/1 [==============================] - 0s 25ms/step


100%|██████████| 6/6 [00:06<00:00,  1.12s/it]


,img_name,img_path,img_folder_path,target,first_top_guess,first_score,second_top_guess,second_score,third_top_guess,third_score
0,892_2_d68b8307-2005-415e-9b5a-1e0029736fee_35c...,/content/data/carpeta1/892_2_d68b8307-2005-415...,carpeta1,0,convertible,0.186214,sports_car,0.161703,racer,0.111139
1,892_11_d68b8307-2005-415e-9b5a-1e0029736fee_51...,/content/data/carpeta1/892_11_d68b8307-2005-41...,carpeta1,0,seat_belt,0.621946,parking_meter,0.052962,backpack,0.032058
2,892_8_d68b8307-2005-415e-9b5a-1e0029736fee_cf0...,/content/data/carpeta1/892_8_d68b8307-2005-415...,carpeta1,0,convertible,0.356400,grille,0.183478,sports_car,0.136136
3,893_11_596264c9-40ee-47c1-a0bb-3cb54ca7eba5_14...,/content/data/carpeta2/893_11_596264c9-40ee-47...,carpeta2,0,convertible,0.328944,seat_belt,0.219503,limousine,0.125315
4,893_9_596264c9-40ee-47c1-a0bb-3cb54ca7eba5_001...,/content/data/carpeta2/893_9_596264c9-40ee-47c...,carpeta2,0,crash_helmet,0.624539,CD_player,0.033826,stove,0.030763
5,893_8_596264c9-40ee-47c1-a0bb-3cb54ca7eba5_059...,/content/data/carpeta2/893_8_596264c9-40ee-47c...,carpeta2,0,odometer,0.503112,radio,0.043996,CD_player,0.036704


In [ ]:
# Creates the csv file
df.to_csv('dataset_after_mobilenet.csv')